In [1]:
import numpy as np
import pandas as pd

# Grouping

![](img/205.png)

![](img/206.png)

![](img/207.png)

![](img/208.png)

Task<br>
- Can you return a table containing the top 10 stores by total transactions in the data?
- Make sure they’re sorted from highest to lowest.

In [2]:
transaction_df = pd.read_csv("data/retail/transactions.csv")
transaction_df

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932


In [8]:
transaction_df.groupby('store_nbr')[['transactions']].sum().sort_values(by='transactions', ascending=False).iloc[:10]

,transactions
store_nbr,
44,7273093
47,6535810
45,6201115
46,5990113
3,5366350
48,5107785
8,4637971
49,4574103
50,4384444


![](img/211.png)

![](img/212.png)

Task <br>
- Can you get me the total transactions by store and month?
- Sort the table from first month to last, then by highest transactions to lowest within each month.

## Multi-Index

![](img/215.png)

![](img/216.png)

![](img/217.png)

![](img/218.png)

Task<br><br>
Can you help me access rows and columns with multiple indices? I’ve been struggling with multi-index DataFrames.

Access:
* Grab Store 3, Month 1
* Then, select the column storing the mean of transactions

Fix:
* Drop the outer layer of the column Index
* Reset the row index so it is the default integer index

In [26]:
# Read in transactions data -- parse dates specified here for help with later problem

transactions = pd.read_csv("data/retail/transactions.csv", parse_dates=["date"])

transactions.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [28]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


In [29]:
transactions["month"] = transactions["date"].dt.month
transactions

,date,store_nbr,transactions,month
0,2013-01-01,25,770,1
1,2013-01-02,1,2111,1
2,2013-01-02,2,2358,1
3,2013-01-02,3,3487,1
4,2013-01-02,4,1922,1
...,...,...,...,...
83483,2017-08-15,50,2804,8
83484,2017-08-15,51,1573,8
83485,2017-08-15,52,2255,8
83486,2017-08-15,53,932,8


In [30]:
grouped = (
    transactions.groupby(["store_nbr", "month"])
    .agg({"transactions": ["sum", "mean"]})
    .sort_values(by=["month", ("transactions", "sum")], ascending=[True, False])
)
grouped

transactions             
                         sum         mean
store_nbr month                          
44        1           628438  4246.202703
47        1           568824  3843.405405
45        1           538370  3637.635135
46        1           522763  3532.182432
3         1           463260  3151.428571
...                      ...          ...
32        12           86167   718.058333
21        12           84128  1402.133333
42        12           76741  1279.016667
29        12           76627  1277.116667
22        12           50650   844.166667

[641 rows x 2 columns]

In [40]:
# Grab store 3, month 1 from multi-index (both values are integers)
grouped.loc[(3,1), :]

transactions  sum     463260.000000
              mean      3151.428571
Name: (3, 1), dtype: float64

In [36]:
# Grab mean column in column multi-index
grouped.loc[: , [('transactions', 'mean')]]

,,transactions
,,mean
store_nbr,month,
44,1,4246.202703
47,1,3843.405405
45,1,3637.635135
46,1,3532.182432
3,1,3151.428571
...,...,...
32,12,718.058333
21,12,1402.133333


In [37]:
# Drop level from column index (axis=1), then reset index
grouped.reset_index().droplevel(0, axis=1)

,,,sum,mean
0,44,1,628438,4246.202703
1,47,1,568824,3843.405405
2,45,1,538370,3637.635135
3,46,1,522763,3532.182432
4,3,1,463260,3151.428571
...,...,...,...,...
636,32,12,86167,718.058333
637,21,12,84128,1402.133333
638,42,12,76741,1279.016667
639,29,12,76627,1277.116667


## Agg Method on Multi-index

![](img/221.png)

![](img/222.png)

![](img/223.png)

![](img/224.png)

Task<br><br>

* Calculate the mean of target met by store, and the sum of bonuses to be paid to each store.

* Sort them by highest to lowest bonus payout.

* Then, do the same for day of week and month.

In [12]:
# Recreate table from section 3

transactions = transactions.assign(
    target_pct=transactions["transactions"] / 2500,
    met_target=(transactions["transactions"] / 2500) >= 1,
    bonus_payable=((transactions["transactions"] / 2500) >= 1) * 100,
    month=transactions.date.dt.month,
    day_of_week=transactions.date.dt.dayofweek,
)

transactions.head()

,date,store_nbr,transactions,month,target_pct,met_target,bonus_payable,day_of_week
0,2013-01-01,25,770,1,0.3080,False,0,1
1,2013-01-02,1,2111,1,0.8444,False,0,2
2,2013-01-02,2,2358,1,0.9432,False,0,2
3,2013-01-02,3,3487,1,1.3948,True,100,2
4,2013-01-02,4,1922,1,0.7688,False,0,2


In [13]:
# Group transactions by store number to get store level stats 
# use agg to calculate mean of met_target, sum of transactions by store
# sort values by bonus payable in descending order


,met_target,bonus_payable
store_nbr,,
47,0.999404,167600
44,0.998807,167500
45,0.997615,167300
3,0.998210,167300
46,0.989267,165900
8,0.888425,148900
48,0.690519,115800
49,0.637448,106900
50,0.453190,76000


In [14]:
# Group transactions by month to get month level stats 
# use agg to calculate mean of met_target, sum of transactions by store
# sort values by bonus payable in descending order


,met_target,bonus_payable
month,,
12,0.255640,154100
5,0.170792,131800
3,0.169461,130400
4,0.174469,129700
7,0.162486,126300
2,0.174230,121700
6,0.161706,121700
8,0.174189,120800
1,0.163723,119600


In [15]:
# Group transactions by day of week to get day of week level stats 
# use agg to calculate mean of met_target, sum of transactions by store
# sort values by bonus payable in descending order



,met_target,bonus_payable
day_of_week,,
5,0.222204,266400
6,0.204001,241700
4,0.179007,213000
0,0.160214,191600
2,0.160572,191000
1,0.146299,175500
3,0.142077,169100


### Transform

![](img/227.png)

# Task<br>
* This time I want a column added to the transactions data that has the average transactions for each store... but I don’t want to lose the rows for each store/day.

* Once you’ve created that column, can you create a new column that contains the difference between the store’s average and its transactions on that day?

## Pivot Table

![](img/230.png)

![](img/231.png)

![](img/232.png)

![](img/233.png)

![](img/234.png)

![](img/235.png)

![](img/236.png)

![](img/237.png)

![](img/238.png)

![](img/239.png)

![](img/240.png)